In [41]:
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt

Some results (e.g. miconic) are either timing out (due to apptainer and cluster cpus) or running out of memory (possibly due to suboptimal handcrafted policies). Again, we should not worry about this too much. I can just increase the resources.

In [42]:
DOMAINS = [
    "blocksworld",
    "ferry",
    "miconic",
    # "rovers",
    "satellite",
    "transport",
]

In [43]:
baseline_data = pd.read_csv('baseline_results.csv')
baseline_data

,domain,problem,repeat,time,plan_length,plan_found,max_bound
0,blocksworld,0_01,0,1.584203,10.0,True,False
1,blocksworld,0_01,1,1.575279,10.0,True,False
2,blocksworld,0_01,2,1.613229,10.0,True,False
3,blocksworld,0_01,3,1.622800,10.0,True,False
4,blocksworld,0_01,4,1.605061,10.0,True,False
...,...,...,...,...,...,...,...
4495,transport,2_30,5,NaN,NaN,False,True
4496,transport,2_30,6,NaN,NaN,False,True
4497,transport,2_30,7,NaN,NaN,False,True
4498,transport,2_30,8,NaN,NaN,False,True


In [44]:
scorpion_data = pd.read_csv("scorpion_results.csv")
scorpion_data

,domain,problem,time,plan_length,plan_found
0,blocksworld,0_01,102.067865,10.0,True
1,blocksworld,0_02,102.641403,8.0,True
2,blocksworld,0_03,103.776869,20.0,True
3,blocksworld,0_04,100.091573,24.0,True
4,blocksworld,0_05,100.256945,24.0,True
...,...,...,...,...,...
248,miconic,2_20,NaN,NaN,False
249,miconic,2_21,NaN,NaN,False
250,miconic,2_22,NaN,NaN,False
251,miconic,2_23,NaN,NaN,False


In [45]:
failure_logs = {domain: [] for domain in DOMAINS}
scorpion_data["solver"] = "scorpion"
baseline_data["solver"] = "baseline"
all_data = pd.concat([baseline_data, scorpion_data])
log_y = True
def plot_domains(metric):
    for domain in DOMAINS:
        print(domain)
        data = all_data[all_data['domain'] == domain]
        failures = data[data.plan_found == False]
        max_bound_failures = data[data.max_bound == False]
        failures_not_due_to_max_bound = data[(data.plan_found == False) & (data.max_bound == False)]
        n_failures = len(failures)
        n_max_bound_failures = len(max_bound_failures)
        n_failures_not_due_to_max_bound = len(failures_not_due_to_max_bound)
        print(f"{n_failures=}")
        for row in failures.itertuples():
            # print(row)
            failure_logs[domain].append(f"/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/{row.domain}_{row.problem}_r{row.repeat}.log")
        fig = px.scatter(data, x="problem", y=metric, color="solver", log_y=log_y)
        fig.show()

In [46]:
plot_domains("plan_length")

blocksworld
n_failures=75


ferry
n_failures=227


miconic
n_failures=670


satellite
n_failures=214


transport
n_failures=78


In [47]:
plot_domains("time")

blocksworld
n_failures=75


ferry
n_failures=227


miconic
n_failures=670


satellite
n_failures=214


transport
n_failures=78


In [48]:
domain = "miconic"
for f in failure_logs[domain]:
    print(f)

/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r0.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r2.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r3.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r4.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r8.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r9.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_05_r0.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_05_r2.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_05_r3.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_05_r4.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs